In [5]:
spark.read
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("voting_data_eng.csv")
    .printSchema

root
 |-- ps_id: integer (nullable = true)
 |-- region_name: string (nullable = true)
 |-- subregion_name: string (nullable = true)
 |-- Baburin Sergei Nikolaevich: integer (nullable = true)
 |-- Grudinin Pavel Nikolaevich: integer (nullable = true)
 |-- Zhirinovskiy Vladimir Volfovich: integer (nullable = true)
 |-- Putin Vladimir Vladimirovich: integer (nullable = true)
 |-- Sobchak Ksenia Anatolyevna: integer (nullable = true)
 |-- Suraikin Maksim Aleksandrovich: integer (nullable = true)
 |-- Titov Boris Yurievich: integer (nullable = true)
 |-- Yavlinskiy Gregory Alekseivich: integer (nullable = true)
 |-- Number of ballot papers in ballot boxes: integer (nullable = true)
 |-- Number of valid ballot papers: integer (nullable = true)
 |-- Number of voters enlisted: integer (nullable = true)
 |-- Number of ballot papers in portable ballot boxes: integer (nullable = true)
 |-- Number of ballot papers given inside the polling station: integer (nullable = true)
 |-- Number of ballot pa

In [59]:
val raw = 
spark.read
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("voting_data_eng.csv")
    .select(
        $"ps_id"
        ,$"region_name"
        ,$"subregion_name"
        ,$"Baburin Sergei Nikolaevich".as("baburin")
        ,$"Grudinin Pavel Nikolaevich".as("grudinin")
        ,$"Zhirinovskiy Vladimir Volfovich".as("zhirinovskiy")
        ,$"Putin Vladimir Vladimirovich".as("putin")
        ,$"Sobchak Ksenia Anatolyevna".as("sobchak")
        ,$"Suraikin Maksim Aleksandrovich".as("suraikin")
        ,$"Titov Boris Yurievich".as("titov")
        ,$"Yavlinskiy Gregory Alekseivich".as("yavlinskiy")
        ,$"Number of ballot papers in ballot boxes".as("papers_in_boxes")
        ,$"Number of valid ballot papers".as("valid_papers")
        ,$"Number of voters enlisted".as("voters")
        ,$"Number of ballot papers in portable ballot boxes".as("papers_portable")
        ,$"Number of ballot papers given inside the polling station".as("papers_inside")
        ,$"Number of ballot papers given outside the polling station".as("papers_outside")
        ,$"Number of ballot papers given for advance voting".as("papers_advance")
        ,$"Number of excessive ballot papers".as("papers_excessive")
        ,$"Number of ballot papers acquired at the polling station".as("papers_acquired")
        ,$"Number of invalid ballot papers".as("papers_invalid")
        ,$"Number of unused ballot papers".as("papers_unused")
        ,$"Number of lost ballot papers".as("papers_lost")
    )
    .withColumn("key",concat($"region_name"))
    .unpersist.cache
raw.limit(10).show
raw.agg(sum($"papers_in_boxes").as("papers_in_boxes")).show
raw.count


+-----+--------------+---------------+-------+--------+------------+-----+-------+--------+-----+----------+---------------+------------+------+---------------+-------------+--------------+--------------+----------------+---------------+--------------+-------------+-----------+--------------+
|ps_id|   region_name| subregion_name|baburin|grudinin|zhirinovskiy|putin|sobchak|suraikin|titov|yavlinskiy|papers_in_boxes|valid_papers|voters|papers_portable|papers_inside|papers_outside|papers_advance|papers_excessive|papers_acquired|papers_invalid|papers_unused|papers_lost|           key|
+-----+--------------+---------------+-------+--------+------------+-----+-------+--------+-----+----------+---------------+------------+------+---------------+-------------+--------------+--------------+----------------+---------------+--------------+-------------+-----------+--------------+
|  713|Altajskij kraj|Bystroistokskaa|      1|     109|          42|  398|      2|       3|    0|         0|          

raw = [ps_id: int, region_name: string ... 22 more fields]


lastException: Throwable = null


94487

In [60]:
//Разбиваю файл на множество файлов по ключу RegionName. Лучше по ps_id+region_name, но для теста сокращу количество файлов
val keys=raw.select($"key").distinct.collect

for(keyRow <- keys ) {
    var key:String = (keyRow.get(0)).asInstanceOf[String]
    var path = s"./ElectionsByKey/$key"
    println(path)
    raw
    .filter($"key".equalTo(lit(key)))
    .coalesce(1)
    .write.format("com.databricks.spark.csv")
    .option("header", "true")
    .mode("overwrite")
    .save(path)
}


Respublika Karelia
./ElectionsByKey/Respublika Karelia
Razanskaa oblast_
./ElectionsByKey/Razanskaa oblast_
Respublika Krym
./ElectionsByKey/Respublika Krym
gorod Sankt-Peterburg
./ElectionsByKey/gorod Sankt-Peterburg
Krasnodarskij kraj
./ElectionsByKey/Krasnodarskij kraj
Magadanskaa oblast_
./ElectionsByKey/Magadanskaa oblast_
Cecenskaa Respublika
./ElectionsByKey/Cecenskaa Respublika
Vologodskaa oblast_
./ElectionsByKey/Vologodskaa oblast_
Kamcatskij kraj
./ElectionsByKey/Kamcatskij kraj
Arhangel_skaa oblast_
./ElectionsByKey/Arhangel_skaa oblast_
Evrejskaa avtonomnaa
./ElectionsByKey/Evrejskaa avtonomnaa
Smolenskaa oblast_
./ElectionsByKey/Smolenskaa oblast_
Vladimirskaa oblast_
./ElectionsByKey/Vladimirskaa oblast_
Voronejskaa oblast_
./ElectionsByKey/Voronejskaa oblast_
Celabinskaa oblast_
./ElectionsByKey/Celabinskaa oblast_
Respublika Ingusetia
./ElectionsByKey/Respublika Ingusetia
Kostromskaa oblast_
./ElectionsByKey/Kostromskaa oblast_
Saratovskaa oblast_
./ElectionsByKey/Sara

keys = Array([Respublika Karelia], [Razanskaa oblast_], [Respublika Krym], [gorod Sankt-Peterburg], [Krasnodarskij kraj], [Magadanskaa oblast_], [Cecenskaa Respublika], [Vologodskaa oblast_], [Kamcatskij kraj], [Arhangel_skaa oblast_], [Evrejskaa avtonomnaa], [Smolenskaa oblast_], [Vladimirskaa oblast_], [Voronejskaa oblast_], [Celabinskaa oblast_], [Respublika Ingusetia], [Kostromskaa oblast_], [Saratovskaa oblast_], [Kurskaa oblast_], [Permskij kraj], [Respublika Buratia], [Respublika Severnaa], [Amalo-Nenezkij avtonomnyj], [Kabardino-Balkarskaa Respublika], [Kirovskaa oblast_], [Kalujskaa oblast_], [Kaliningradskaa oblast_], [Cukotskij avtonomnyj], [Respublika Komi], [Belgorodskaa oblast_], [Amurskaa oblast_], [Respublika Baskortostan], [Omskaa oblast...


Array([Respublika Karelia], [Razanskaa oblast_], [Respublika Krym], [gorod Sankt-Peterburg], [Krasnodarskij kraj], [Magadanskaa oblast_], [Cecenskaa Respublika], [Vologodskaa oblast_], [Kamcatskij kraj], [Arhangel_skaa oblast_], [Evrejskaa avtonomnaa], [Smolenskaa oblast_], [Vladimirskaa oblast_], [Voronejskaa oblast_], [Celabinskaa oblast_], [Respublika Ingusetia], [Kostromskaa oblast_], [Saratovskaa oblast_], [Kurskaa oblast_], [Permskij kraj], [Respublika Buratia], [Respublika Severnaa], [Amalo-Nenezkij avtonomnyj], [Kabardino-Balkarskaa Respublika], [Kirovskaa oblast_], [Kalujskaa oblast_], [Kaliningradskaa oblast_], [Cukotskij avtonomnyj], [Respublika Komi], [Belgorodskaa oblast_], [Amurskaa oblast_], [Respublika Baskortostan], [Omskaa oblast...